<a href="https://colab.research.google.com/github/serasset/llod-cloud-evaluator/blob/main/BATS_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Transforming BATS to RDF

Example of BATS structure: 
```
acrobat	troupe
antelope	herd
bee	swarm/hive/colony/drift/cast/nest/rabble/stand
bird	flock
book	library/shelf/collection/accounting/accounting_system/method_of_accounting
````

Modeling in RDF, using Ontolex:

```
PREFIX...

:acrobat a ontolex:LexicalEntry;
  rdfs:label "acrobat"@en.

:troupe a ontolex:LexicalEntry;
  rdfs:label "troupe"@en.

:meroMemember_01 a rdf:Relation;
  rdf:source :acrobat ;
  rdf:target :troupe ;
  rdf:property bats:meronym_member ;
  bats:rank "1"@int .

```


In [ ]:
%%capture 
!pip3 install rdflib 

In [ ]:
import sys
import Wrapping_SPARQL_Python.ipynb as wrapper
from rdflib import URIRef, Literal, Namespace
from rdflib.namespace import RDFS, RDF

command_args = sys.argv[1:]
filename = command_args[0]


# TODO: handle file spec
with open(filename) as f:
    
    for line in f:
      
      terms = line.split()
      lead_word = terms[0]
      associated_terms = terms.split('/')

      BATS = Namespace("https://vecto.space/projects/BATS/")

      ONTOLEX = Namespace("http://www.w3.org/ns/lemon/ontolex#")
      

      entry_subj = URIRef(BATS + "/{lead_word}")

      g = Graph()
      g.bind(BATS)

      # Creates entry word entry
      g.add((entry_subj, RDF.type, ONTOLEX.LexicalEntry))
      g.add((entry_subj, RDFS.label, Literal(lead_word, lang="en")))

      # Creates an ontloex entry and an RDF relation for each associated term
      for count in range(len(associated_terms)):

        curr_word = associated_terms[count]
        curr_subj = URIRef(BATS + "/{curr_word}")

        g.add((curr_subj, RDF.type, ONTOLEX.LexicalEntry))
        g.add((curr_subj, RDFS.label, Literal(curr_word, lang = "en"))

        # Maintains the rank of the current associated word
        rank = str(count + 1).zfill(2)
        mero_mem_subj = URIRef(".../meroMember_{rank}")
        
        g.add((mero_mem_subj, RDF.type, RDF.Relation))
        g.add((mero_mem_subj, RDF.source, entry_subj))
        g.add((mero_mem_subj, RDF.target, curr_subj))
        g.add((mero_mem_subj, RDF.property, BATS.meronym_member))

      wrapper.visualize(g)

   # TODO: what to do with serialized graphs?






      
